### Master Trust/Warehousing Funding Example

In [553]:
from absbox import Generic,API,EnginePath,readBondsCf,readFeesCf

test01 = Generic(
    "TEST01"
    ,{"collect": ["2024-04-18","2024-05-01"] #collection_dates: next pool colle
    ,"pay": ["2024-04-19"   #payment_dates last distribution payment date, next 
            ,"2024-05-03"]  
    ,"stated":"2025-11-16"
    ,"poolFreq":"MonthEnd"
    ,"payFreq":["DayOfMonth",20]
    }
    ,{'assets':[["Mortgage"
        ,{"originBalance":100000,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2024-01-01"}
          ,{"currentBalance":100000
          ,"currentRate":0.08
          ,"remainTerm":26
          ,"status":"current"}]],'issuanceStat': {'IssuanceBalance': 2000.86}}
    ,(('collections_interest', {'balance': 0}), ('collections_principal', {'balance': 100000}), ('revolBuyAcc', {'balance': 0}), ('reserveAccount', {'balance': 1000000, 'type': {'targetReserve': [('poolBalance',), 0.0125]}}))
    ,(("A",{"A1":{"balance": 10_000_000.84 
                    ,"rate":0.1023
                    ,"originBalance":10_000_000
                    ,"originRate":0.1023  
                    ,"startDate":"2023-07-03"
                    ,"rateType":{"fix": 0.1023, "dayCount":"DC_ACT_365A"}
                    ,"maturityDate":"2025-11-16"
                    ,"bondType":{"Lockout":"2023-11-04"}
                    }
                    }
      )
      ,("B",{"balance":1
                    ,"rate":0.0
                    ,"originBalance":1
                    ,"originRate":0.0
                    ,"startDate":"2023-07-03"
                    ,"maturityDate":"2025-11-16"
                    ,"rateType":{"Fixed":0.00}
                    ,"bondType":{"Equity":None}
                    }
                    ))
    ,(("technology_fee",{"type":{"annualPctFee":[("bondBalance",),0.0005]}}),
      ("structuring_fee",{"type":{"annualPctFee":[("originalBondBalance",),0.0075]}}),
      ("servicing_fee",{"type":{"annualPctFee":[("bondBalance","A"),0.01]}}),
      ("security_fee",{"type": {"recurFee":[["MonthDayOfYear",5,16],30000]}})
     )
    ,{
        "default":[       
              ["IfElse" ,["status","Revolving"] # actions in the revolving period
                ,[["calcIntByGroup",["A"]],
              ["calcAndPayFee","collections_interest",["technology_fee","structuring_fee","security_fee"]]
                ,["accrueAndPayIntByGroup","collections_interest","A","byStartDate"]
                ,["payIntByGroup","collections_principal","A","byStartDate"] # If interest account has no money then use collections to pay the remaining
                
                ,["transfer","collections_interest",'reserveAccount',{"reserve":"gap"}]
                ,["transfer","collections_principal",'reserveAccount',{"reserve":"gap"}]
                
                
                #fourth cure LTV breach.
                
                ,["calcAndPayFee","collections_interest",["servicing_fee"]] 
                ,["payIntResidual","collections_interest","B"]
                
                ,["transfer" ,"collections_principal","revolBuyAcc"]
                ,["buyAsset",["Current|Defaulted",1.0,0],"revolBuyAcc",None]                  

                  ]
                ,[["transfer" ,"collections_interest",'collections_principal'],
                   ["transfer" ,"reserveAccount",'collections_principal'],
                  ["calcIntByGroup",["A"]], # actions if deal is in Amortizing status 
              ["calcAndPayFee","collections_principal",["technology_fee","structuring_fee","security_fee","servicing_fee"]]
              ,["accrueAndPayIntByGroup","collections_principal","A","byStartDate"]
                  ,["payPrinByGroup","collections_principal","A","byStartDate"]
                  ,["payIntResidual","collections_principal","B"]
              ,["calcAndPayFee","collections_principal",["servicing_fee"]]
                   ,["payPrin","collections_principal",["B"]]

                  ]]
              ]
        #,"endOfCollection":[["payFee","collections",["technology_fee","structuring_fee","security_fee"]]  # accure fee by end of collection period
         #                     ,["calcInt","A1"]]
        ,"cleanUp":[["sellAsset", ["Current|Defaulted",1.0,0], "collections_principal"]
                    ,["transfer" ,"reserveAccount",'collections_principal']
                    ,["transfer" ,"revolBuyAcc",'collections_principal']
                    ,["accrueAndPayIntByGroup","collections_interest","A","byStartDate"]
                    ,["payIntResidual","collections_interest","B"]
                    ,["transfer" ,"collections_interest",'collections_principal']
                    ,["payPrinByGroup","collections_principal","A","byStartDate"]
                    ,["payPrin","collections_principal",["B"]]
                    ]
        }
    ,[["CollectedPrincipal","collections_principal"]
      ,["CollectedRecoveries","collections_principal"]
      ,["CollectedPrepayment","collections_principal"]
      ,["CollectedInterest","collections_interest"]
     ]
    ,None
    ,None
    ,None
    ,None
    ,("Amortizing")
    )


fundingPlan = [("2024-10-01","A","collections_principal"
                    ,{"balance":1500000
                       ,"rate":0.1023
                       ,"name":"A2"
                       ,"originBalance":1500000
                       ,"originRate":0.1023
                       ,"rateType":{"Fixed":0.1023}
                       ,"bondType":{"Lockout":"2024-10-04"}
                      ,"maturityDate":"2025-11-16"}
                    )]

localAPI = API(EnginePath.DEV,lang='english')
r = localAPI.run(test01
                 ,runAssump = [
                     ("issueBond",*fundingPlan)
                     ,("stop","2024-11-21")
                 ]
                 ,read=True)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.28.7, server:0.28.15

Warning Message from server:
Bond A1 is not paid off
Bond A2 is not paid off
Bond B is not paid off
Fee servicing_fee is not paid off
Fee structuring_fee is not paid off
Fee technology_fee is not paid off
No waterfall distribution found on date2024-05-03with statusAmortizing
No waterfall distribution found on date2024-05-20with statusAmortizing
No waterfall distribution found on date2024-06-20with statusAmortizing
No waterfall distribution found on date2024-07-20with statusAmortizing
No waterfall distribution found on date2024-08-20with statusAmortizing
No waterfall distribution found on date2024-09-20with statusAmortizing
No waterfall distribution found on date2024-10-20with statusAmortizing
No waterfall distribution found on date2024-11-20with statusAmortizing

In [554]:
readBondsCf(r['bonds'])

BondGroup            A                                                        \
Bond                A1                                                    A2   
Field          balance   interest   principal    rate        cash    balance   
date                                                                           
2024-05-03  8946143.29   39238.35  1053857.55  0.1023  1093095.90        NaN   
2024-05-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-06-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-07-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-08-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-09-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-10-20  7973663.99  426253.06   972479.30  0.1023  1398732.36  1500000.0   
2024-11-20  7973663.99       0.00        0.00  0.1023        0.00  1500000.0   

BondGroup                                           B                          \
Bond                                                -                           
Field      interest principal    rate    cash balance interest principal rate   
date                                                                            
2024-05-03      NaN       NaN     NaN     NaN       1        0         0    0   
2024-05-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-06-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-07-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-08-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-09-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-10-20   7987.8       0.0  0.1023  7987.8       1        0         0    0   
2024-11-20      0.0       0.0  0.1023     0.0       1        0         0    0   

BondGroup        
Bond             
Field      cash  
date             
2024-05-03    0  
2024-05-20    0  
2024-06-20    0  
2024-07-20    0  
2024-08-20    0  
2024-09-20    0  
2024-10-20    0  
2024-11-20    0

In [516]:
# technology fee
## 2024-5-3
1/2000 * (8946143.29+1053857.55) * (14/365)  # 191.7808380273973

191.7808380273973

In [517]:
# technology fee
## 2024-5-20
1/2000 * (8946143.29) * (17/365)

208.33484373972604

In [518]:
# technology fee
## 2024-6-20
1/2000 * (8946143.29) * (31/365), 208.33484373972604 + 1/2000 * (8946143.29) * (31/365)

(379.9047150547945, 588.2395587945206)

In [519]:
# technology fee
## 2024-7-20
1/2000 * (8946143.29) * (30/365)+ 588.23 - 60.78

895.0997242465753

In [520]:
# servicing_fee 
## 2024-5-3
(8946143.29+1053857.55)*(14/365)*0.01

3835.6167605479454

In [521]:
# servicing_fee 
## 2024-5-20
(8946143.29)*(17/365)*0.01

4166.69687479452

In [522]:
# servicing_fee 
## 2024-6-20
(8946143.29)*(31/365)*0.01+4166.69687

11764.791171095889

In [523]:
# servicing_fee
## 2024-7-20
8946143.29*(30)/365*0.01 + 11764.791171095889 - 1215.25

17902.535656027394

In [524]:
# servicing_fee
# 2024-10-20
(1500000.0) * (19/365) * 0.01 + 8946143.29 * (30/365) * 0.01

8133.816402739724

In [525]:
38285.77 - 30151.96

8133.809999999998

In [543]:
# servicing_fee
# 2024-11-20
7973663.99 * (31/365) * 0.01 ,(1500000.0) * (31/365)* 0.01 

(6772.1529778082195, 1273.972602739726)

In [544]:
(6772.1529778082195+1273.972602739726)

8046.125580547945

In [545]:
5902.79+2143.33 # total due

8046.12

In [546]:
# Technoloy fee
## 2024-11-20
(1500000.0+7974003.03) * (31/365)* 0.0005

402.3206766164384

In [547]:
295.14+107.18

402.32

In [531]:
readFeesCf(r['fees'])

Fee        security_fee           servicing_fee           structuring_fee  \
Field           balance   payment       balance   payment         balance   
date                                                                        
2024-05-03         0.00      0.00          0.00   3835.61            0.00   
2024-05-20     30000.00      0.00       4166.69      0.00         3493.15   
2024-06-20     30000.00      0.00      11764.78      0.00         9863.01   
2024-07-20     28093.00   1907.00      17902.52   1215.25        15008.58   
2024-08-20     26544.85   1548.15      24095.32   1405.29        20200.31   
2024-09-20     25253.81   1291.04      30151.96   1541.45        25277.90   
2024-10-20         0.00  25253.81          0.00  38285.77            0.00   
2024-11-20         0.00      0.00       5902.79   2143.33         5374.01   

Fee                  technology_fee           
Field        payment        balance  payment  
date                                          
2024-05-03   2876.71           0.00   191.78  
2024-05-20      0.00         208.33     0.00  
2024-06-20      0.00         588.23     0.00  
2024-07-20   1018.81         895.09    60.78  
2024-08-20   1178.13        1204.71    70.28  
2024-09-20   1292.27        1507.52    77.09  
2024-10-20  32027.90           0.00  1914.21  
2024-11-20   1951.33         295.12   107.18

### New Fee



In [556]:
newFee = ("uncommited_fee"
          ,{"type": 
            {"annualPctFee": [("excess",("originalBondBalance", "A"),("bondBalance", "A"),),0.01],}
            ,"feeStart": "2024-02-21"})

In [557]:
from lenses import lens

newTest01 = test01 & lens.fees.modify(lambda xs: xs + (newFee,))

feeNames = [h for h,_ in newTest01.fees]

newTest01 &=  lens.waterfall['default'][0][2][1][2].set(feeNames)
newTest01 &=  lens.waterfall['default'][0][3][3][2].set(feeNames)

In [558]:
feeNames

['technology_fee',
 'structuring_fee',
 'servicing_fee',
 'security_fee',
 'uncommited_fee']

In [559]:
r2 = localAPI.run(newTest01
                 ,runAssump = [
                     ("issueBond",*fundingPlan)
                     ,("stop","2024-11-21")
                 ]
                 ,read=True)

Warning Message from server:
Bond A1 is not paid off
Bond A2 is not paid off
Bond B is not paid off
Fee servicing_fee is not paid off
Fee structuring_fee is not paid off
Fee technology_fee is not paid off
Fee uncommited_fee is not paid off
No waterfall distribution found on date2024-05-03with statusAmortizing
No waterfall distribution found on date2024-05-20with statusAmortizing
No waterfall distribution found on date2024-06-20with statusAmortizing
No waterfall distribution found on date2024-07-20with statusAmortizing
No waterfall distribution found on date2024-08-20with statusAmortizing
No waterfall distribution found on date2024-09-20with statusAmortizing
No waterfall distribution found on date2024-10-20with statusAmortizing
No waterfall distribution found on date2024-11-20with statusAmortizing

In [476]:
readFeesCf(r2['fees'])

Fee        security_fee           servicing_fee           structuring_fee  \
Field           balance   payment       balance   payment         balance   
date                                                                        
2024-05-03         0.00      0.00          0.00   3835.61            0.00   
2024-05-20     30000.00      0.00       4166.69      0.00         3493.15   
2024-06-20     30000.00      0.00      11764.78      0.00         9863.01   
2024-07-20     28155.83   1844.17      17942.56   1175.21        15042.15   
2024-08-20     26665.72   1490.11      24188.94   1351.71        20278.81   
2024-09-20     25426.77   1238.95      30310.13   1476.90        25410.51   
2024-10-20         0.00  25426.77          0.00  38443.94            0.00   
2024-11-20         0.00      0.00       6116.82   1933.47         5565.99   

Fee                  technology_fee          uncommited_fee           
Field        payment        balance  payment        balance  payment  
date                                                                  
2024-05-03   2876.71           0.00   191.78           0.00     0.00  
2024-05-20      0.00         208.33     0.00         490.83     0.00  
2024-06-20      0.00         588.23     0.00        1385.88     0.00  
2024-07-20    985.24         897.08    58.79        2113.63   138.43  
2024-08-20   1133.20        1209.38    67.60        2849.45   159.23  
2024-09-20   1238.16        1515.41    73.87        3570.53   173.97  
2024-10-20  32160.51           0.00  1922.10           0.00  4436.71  
2024-11-20   1759.35         305.82    96.69        1304.49   412.33

In [561]:
r2['fees']['uncommited_fee']

,balance,payment,due
date,,,
2024-05-03,0.00,0.00,0
2024-05-20,490.83,0.00,0
2024-06-20,1385.88,0.00,0
2024-07-20,2113.63,138.43,0
2024-08-20,2849.45,159.23,0
2024-09-20,3570.53,173.97,0
2024-10-20,0.00,4436.71,0
2024-11-20,1304.49,412.33,0


In [562]:
readBondsCf(r2['bonds'])

BondGroup            A                                                        \
Bond                A1                                                    A2   
Field          balance   interest   principal    rate        cash    balance   
date                                                                           
2024-05-03  8946143.29   39238.35  1053857.55  0.1023  1093095.90        NaN   
2024-05-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-06-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-07-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-08-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-09-20  8946143.29       0.00        0.00  0.1023        0.00        NaN   
2024-10-20  7978572.33  426253.06   967570.96  0.1023  1393824.02  1500000.0   
2024-11-20  7978572.33       0.00        0.00  0.1023        0.00  1500000.0   

BondGroup                                           B                          \
Bond                                                -                           
Field      interest principal    rate    cash balance interest principal rate   
date                                                                            
2024-05-03      NaN       NaN     NaN     NaN       1        0         0    0   
2024-05-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-06-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-07-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-08-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-09-20      NaN       NaN     NaN     NaN       1        0         0    0   
2024-10-20   7987.8       0.0  0.1023  7987.8       1        0         0    0   
2024-11-20      0.0       0.0  0.1023     0.0       1        0         0    0   

BondGroup        
Bond             
Field      cash  
date             
2024-05-03    0  
2024-05-20    0  
2024-06-20    0  
2024-07-20    0  
2024-08-20    0  
2024-09-20    0  
2024-10-20    0  
2024-11-20    0

In [563]:
r2['accounts']['collections_principal'].loc["2024-10-20"]

,balance,change,memo
date,,,
2024-10-20,1504201.85,595.42,"<Transfer:collections_interest,collections_pri..."
2024-10-20,1504201.85,0.00,"<Transfer:reserveAccount,collections_principal>"
2024-10-20,1401811.82,-102390.03,<SeqPayFee:technology_feestructuring_feeservic...
2024-10-20,967570.96,-434240.86,<PayInt:A>
2024-10-20,0.00,-967570.96,<PayPrin:A>
2024-10-20,0.00,0.00,<PayYield:B>
2024-10-20,0.00,0.00,<SeqPayFee:servicing_fee>
2024-10-20,0.00,0.00,<PayPrin:B>


In [423]:
# 2024-5-20
10_000_000 * (17)/365 * 0.01 - 8946143.29 * 17/365 * 0.01 

490.83737178082174

In [424]:
# 2024-6-20
(10_000_000 - 8946143.29) * 31/365 * 0.01 + 490.83

1385.8863838356172

In [425]:
# 2024-7-20
(10_000_000 - 8946143.29) * 30/365 * 0.01 + 1385.8863838356172 - 138.43

2113.639981095892

In [510]:
# 2024-10-20
(10_000_000 * 30/365 * 0.01 ) - (8946143.29 * 30/365 * 0.01 )+3570.53

4436.7135972602755

In [429]:
# 2024-11-20 - Original balance
(10_000_000 * 31/365) + (1_500_000* 31/365 )

976712.3287671233

In [430]:
# current balance
(8946143.29 * 31/365) +(1500000.0 * 31/365)

887206.6903835615

In [512]:
0.01 * ((10_000_000) - (7978572.33)) * 31/365

1716.8289799999998

In [513]:
1304.49 + 412.33

1716.82